In [ ]:
from pathlib import Path
from pypdf import PdfReader
import pandas as pd
import re

# -------- configuración --------
BASE_DIR = Path("C:/Users/Rodrigo/Desktop/Rodrigo proyects/nlp-candidates")
assert BASE_DIR.exists(), f"No existe: {BASE_DIR}"
OUT_DIR = Path.cwd()
OUT_DIR.mkdir(exist_ok=True)

# -------- utilidades --------
def normalizar(x):
    if not x: return ""
    x = x.replace("\x00","")
    x = re.sub(r"[ \t]+"," ", x)
    x = re.sub(r"\s+\n","\n", x)
    x = re.sub(r"\n{3,}","\n\n", x)
    return x.strip()

def candidato_desde_nombre(p):
    return p.stem.lower()

def leer_pdf_por_pagina(p):
    try:
        r = PdfReader(str(p))
        out = []
        for i in range(len(r.pages)):
            t = r.pages[i].extract_text() or ""
            out.append(t)
        return out
    except Exception as e:
        print("ERROR leyendo", p.name, "->", e)
        return []

# -------- extracción --------
rows = []
pdfs = sorted(BASE_DIR.glob("*.pdf"))
if not pdfs:
    raise SystemExit("No se encontraron PDFs en la carpeta especificada.")

for p in pdfs:
    pags = leer_pdf_por_pagina(p)
    for i, txt in enumerate(pags, start=1):
        rows.append({
            "candidate": candidato_desde_nombre(p),
            "filename": p.name,
            "page": i,
            "text": normalizar(txt)
        })

pages = pd.DataFrame(rows)
if pages.empty:
    raise SystemExit("No se extrajo texto. Revisa si los PDFs son escaneados (imagen).")

# métricas rápidas (útil para QC)
pages["n_chars"] = pages["text"].str.len()
pages["n_words"] = pages["text"].str.split().apply(len)

# -------- agregación por documento --------
docs = (pages
        .sort_values(["filename","page"])
        .groupby(["candidate","filename"], as_index=False)["text"]
        .apply(lambda s: "\n\n".join(s.values))
       )
docs["n_chars"] = docs["text"].str.len()
docs["n_words"] = docs["text"].str.split().apply(len)

# -------- persistencia en parquet y csv --------
pages_parquet = OUT_DIR / "pages.parquet"
docs_parquet  = OUT_DIR / "documents.parquet"
pages_csv     = OUT_DIR / "pages.csv"
docs_csv      = OUT_DIR / "documents.csv"

pages.to_parquet(pages_parquet, index=False)
docs.to_parquet(docs_parquet, index=False)

# CSV en UTF-8 con BOM por compatibilidad amplia en Windows/Excel
pages.to_csv(pages_csv, index=False, encoding="utf-8-sig")
docs.to_csv(docs_csv, index=False, encoding="utf-8-sig")

print("Listo.")
print("Archivos generados:")
print(" -", pages_parquet)
print(" -", docs_parquet)
print(" -", pages_csv)
print(" -", docs_csv)

# -------- reporte rápido de PDFs sospechosos (posibles escaneos sin texto) --------
print("\nPosibles PDFs con poco texto (revisar OCR si aparecen arriba):")
print(pages.groupby("filename")["n_words"].sum().sort_values().head(10))
